In [1]:
""" Wikipedia Search Engine Notebook

Εισαγωγή
Σε αυτό το Notebook, υλοποιούμε μία μηχανή αναζήτησης που βασίζεται σε δεδομένα από τη Wikipedia.
Ο κώδικας περιλαμβάνει:
1. Συλλογή δεδομένων από τη Wikipedia.
2. Επεξεργασία κειμένου και δημιουργία αντιστραμμένου ευρετηρίου.
3. Υλοποίηση βασικών λειτουργιών αναζήτησης.
4. Αξιολόγηση της μηχανής αναζήτησης.

Βιβλιοθήκες και Ρυθμίσεις
Εδώ εισάγουμε τις απαραίτητες βιβλιοθήκες για τη συλλογή και επεξεργασία των δεδομένων.
Κατεβάζουμε επίσης τα απαραίτητα δεδομένα για την επεξεργασία κειμένου."""


import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import string
import requests
from bs4 import BeautifulSoup
import os
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Κατεβάζει τα απαραίτητα δεδομένα για tokenization και stopwords
nltk.download('punkt')
nltk.download('stopwords')

class WikipediaScraper:
    """Κλάση για συλλογή δεδομένων από την Wikipedia."""
    
    def __init__(self, base_url, csv_file):
        """
        Αρχικοποιεί τη WikipediaScraper.
        :param base_url: Η βασική διεύθυνση URL της Wikipedia.
        :param csv_file: Το όνομα του αρχείου CSV για αποθήκευση δεδομένων.
        """
        self.base_url = base_url
        self.csv_file = csv_file

    def scrape(self, topic_list):
        """
        Συλλέγει άρθρα από τη Wikipedia βάσει των θεμάτων και τα αποθηκεύει σε CSV.
        :param topic_list: Λίστα με τα θέματα προς αναζήτηση στη Wikipedia.
        """
        if os.path.exists(self.csv_file):
            # Αν το αρχείο υπάρχει ήδη, αποφεύγεται η συλλογή δεδομένων
            print(f"File {self.csv_file} already exists. Skipping scraping.")
            return

        with open(self.csv_file, 'w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['doc_id', 'content'])  # Εισάγει κεφαλίδα στο αρχείο
            doc_id = 1

            for topic in topic_list:
                # Δημιουργεί το URL για κάθε θέμα και κατεβάζει το περιεχόμενο
                url = f"{self.base_url}{topic}"
                print(f"Scraping: {url}")
                response = requests.get(url)
                if response.status_code == 200:
                    # Εξάγει το περιεχόμενο των παραγράφων με BeautifulSoup
                    soup = BeautifulSoup(response.content, 'html.parser')
                    paragraphs = soup.find_all('p')
                    content = " ".join([para.get_text() for para in paragraphs])
                    if content.strip():  # Αποθηκεύει το περιεχόμενο αν δεν είναι κενό
                        writer.writerow([doc_id, content])
                        doc_id += 1
                else:
                    print(f"Failed to retrieve: {url}")

        print(f"Scraping complete. Data saved to {self.csv_file}")


class SearchEngine:
    """Κλάση μηχανής αναζήτησης για διαχείριση και αναζήτηση εγγράφων."""
    
    def __init__(self, csv_file):
        """
        Αρχικοποιεί τη μηχανή αναζήτησης.
        :param csv_file: Το όνομα του αρχείου CSV που περιέχει τα έγγραφα.
        """
        self.csv_file = csv_file
        self.documents = {}
        self.inverted_index = defaultdict(list)
        self.stop_words = set(stopwords.words('english'))  # Φορτώνει τις stopwords
        self.stemmer = PorterStemmer()  # Stemmer για την κανονικοποίηση
        self.lemmatizer = WordNetLemmatizer()  # Lemmatizer για την κανονικοποίηση
        self._load_documents()  # Φορτώνει τα έγγραφα από το CSV
        self._build_inverted_index()  # Δημιουργεί το inverted index

    def _load_documents(self):
        """
        Φορτώνει τα έγγραφα από το αρχείο CSV και τα αποθηκεύει στο dictionary.
        """
        if not os.path.exists(self.csv_file):
            raise FileNotFoundError(f"File {self.csv_file} does not exist.")

        with open(self.csv_file, 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Παραλείπει την κεφαλίδα
            for row in reader:
                doc_id, content = row[0], row[1]
                self.documents[doc_id] = content

    def _preprocess(self, text):
        """
        Κανονικοποιεί το κείμενο με αφαίρεση ειδικών χαρακτήρων, tokenization, 
        φιλτράρισμα stopwords, και stemming/lemmatization.
        :param text: Το κείμενο προς επεξεργασία.
        :return: Λίστα κανονικοποιημένων tokens.
        """
        text = ''.join(char for char in text if char.isalnum() or char.isspace())  # Αφαίρεση ειδικών χαρακτήρων
        tokens = word_tokenize(text.lower())  # Tokenization και μετατροπή σε μικρά γράμματα
        tokens = [token for token in tokens if token not in self.stop_words]  # Αφαίρεση stopwords
        tokens = [self.stemmer.stem(self.lemmatizer.lemmatize(token)) for token in tokens]  # Εφαρμογή stemming και lemmatization
        return tokens

    def _build_inverted_index(self):
        """
        Δημιουργεί το inverted index χρησιμοποιώντας τα tokens των εγγράφων.
        """
        for doc_id, content in self.documents.items():
            tokens = self._preprocess(content)
            for token in set(tokens):  # Χρησιμοποιεί μοναδικά tokens για αποφυγή διπλών εισαγωγών
                self.inverted_index[token].append(doc_id)

    def search(self, query):
        """
        Εκτελεί αναζήτηση στα έγγραφα βάσει του ερωτήματος.
        :param query: Το ερώτημα αναζήτησης.
        :return: Λίστα doc_ids που ταιριάζουν με το ερώτημα.
        """
        query_tokens = self._preprocess(query)
        if not query_tokens:
            return []

        doc_scores = defaultdict(int)  # Αποθηκεύει τις βαθμολογίες εγγράφων
        for token in query_tokens:
            if token in self.inverted_index:
                for doc_id in self.inverted_index[token]:
                    doc_scores[doc_id] += 1

        # Επιστρέφει τα έγγραφα με ταξινόμηση βάσει της βαθμολογίας
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        return [doc_id for doc_id, _ in sorted_docs]

    def evaluate(self, test_queries, test_labels):
        """
        Αξιολογεί τη μηχανή αναζήτησης χρησιμοποιώντας μετρικές precision, recall, και F1-score.
        :param test_queries: Λίστα ερωτημάτων αναζήτησης.
        :param test_labels: Λίστα με λίστες σχετικών doc_ids για κάθε ερώτημα.
        :return: Precision, recall, και F1-score.
        """
        y_true = []
        y_pred = []

        for query, relevant_docs in zip(test_queries, test_labels):
            retrieved_docs = self.search(query)
            y_true.extend([1 if doc in relevant_docs else 0 for doc in self.documents.keys()])
            y_pred.extend([1 if doc in retrieved_docs else 0 for doc in self.documents.keys()])

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        return precision, recall, f1


if __name__ == "__main__":
    print("Enter Wikipedia topics separated by commas (e.g., Natural_language_processing,Python_(programming_language)):")
    user_input = input("Topics: ")
    topics = [topic.strip() for topic in user_input.split(',') if topic.strip()]  # Επεξεργασία θεμάτων

    if not topics:
        print("No topics provided. Exiting.")
        exit()

    scraper = WikipediaScraper("https://en.wikipedia.org/wiki/", "wikipedia_documents.csv")
    scraper.scrape(topics)

    search_engine = SearchEngine('wikipedia_documents.csv')

    test_queries = ["machine learning", "neural networks"]
    test_labels = [["1", "2"], ["3", "4"]]  # Δείγματα για αξιολόγηση
    precision, recall, f1 = search_engine.evaluate(test_queries, test_labels)
    print(f"Evaluation Results - Precision: {precision}, Recall: {recall}, F1-score: {f1}")

    while True:
        query = input("Enter your search query (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        results = search_engine.search(query)
        if results:
            print("Documents found:")
            for doc_id in results:
                print(f"- {doc_id}: {search_engine.documents[doc_id][:1000]}...")
        else:
            print("No documents found.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter Wikipedia topics separated by commas (e.g., Natural_language_processing,Python_(programming_language)):
Topics: python,C,C++
Scraping: https://en.wikipedia.org/wiki/python
Scraping: https://en.wikipedia.org/wiki/C
Scraping: https://en.wikipedia.org/wiki/C++
Scraping complete. Data saved to wikipedia_documents.csv
Evaluation Results - Precision: 0.5, Recall: 0.3333333333333333, F1-score: 0.4
Enter your search query (or type 'exit' to quit): coding
Documents found:
- 2: 
 C or c is the third letter of the Latin alphabet, used in the modern English alphabet, the alphabets of other western European languages and others worldwide. Its name in English is cee (pronounced /ˈsiː/), plural cees.[1]
 "C" comes from the same letter as "G". The Semites named it gimel. The sign is possibly adapted from an Egyptian hieroglyph for a staff sling, which may have been the meaning of the name gimel. Another possibility is that it depicted a camel, the Semitic name for which was gamal. Barry B. Powel